# Project 2, APS1070 Summer 2020
**Anomaly Detection Algorithm using Gaussian Mixture Model [20 Marks]**

**Deadline: June 18, 23:00 - 20 points**

**Academic Integrity**

This project is individual - it is to be completed on your own. If you have questions, please post your query in the APS1070 Piazza Q&A forums (the answer might be useful to others!).

Do not share your code with others, or post your work online. Do not submit code that you have not written yourself. Students suspected of plagiarism on a project, midterm or exam will be referred to the department for formal discipline for breaches of the Student Code of Conduct.

Please fill out the following:


*   Name: ##
*   Student Number: ##

##Part 1: Getting started [6 Marks]

We are going to work with a credit card fraud dataset. This dataset contains 28 key features, which are not 
directly interpretable but contain meaningful information about the dataset.

Load the dataset in CSV file using Pandas. The dataset is called `creditcard.csv`. Print out the first few columns of the dataset.

* How many rows are there? _____ **284807**
* What features in the dataset are present aside from the 28 main features?  _____ **Amount, class and Time**
* Which column contains the targets? To what do the values correspond?_____ **the class column, (validate the dataset like 'features')**

In [2]:
pip install wget


Note: you may need to restart the kernel to use updated packages.


In [3]:
import wget


wget.download('https://github.com/aps1070-2019/datasets/raw/master/creditcard.tar.gz','creditcard.tar.gz')

URLError: <urlopen error [Errno 11004] getaddrinfo failed>

In [ ]:
!tar -zxvf creditcard.tar.gz

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
df = pd.read_csv('creditcard.csv')
df['rows']= np.arange(0,df['V23'].values.shape[0],1)


In [ ]:
### HERE ###
count_0 = 0
count_1 = 0

df

It's important when looking at a new dataset to figure out how many examples we have for each class.

* What is the percentage of entries in the dataset for each class? _____**99.827%**
* Is this data considered balanced or unbalanced? Why is this the case?_____**I think it's unbalanced, because the dataset contains too little of the positive targets**
* Why is balance/imbalance important? How might this class ditribution affect a KNN classifier for example, which we explored in Project 1? _____**it may cause a problem if the positive label is not enough. the KNN classifier might be underfitted** 



In [ ]:
class_count = df['Class'].value_counts()
pctg = class_count[0]/(class_count[0]+class_count[1])
print(pctg)


Next, split the dataset into a training and testing set. Use a test size of one third, and set the random state to 1.

Make sure to separate out the column corresponding to the targets.

As mentioned earlier, in this lab we are going to use Gaussian distributions to model the data. To accomplish this, we are going to introduce `scipy`, a package which contains a wide variety of tools for working with scientific data in Python. Its `stats` package allows us to easily model various statistical distributions, and get information about them.

Scipy's Gaussian distribution class is called `norm`. It takes two parameters - `loc`, which corresponds to the mean of your distribution, and `scale`, which corresponds to the standard deviation.

* What are the mean and standard deviation for variable V11? Make sure to only use your training set for this calculation. ___**-0.0009844107658064413 1.0199687920065503**__

Use the code below to set up a Gaussian object for V11.

In [ ]:
### Split the data  ###
#X_train, X_test, y_train, y_test = 0, 0, 0, 0

In [ ]:
from scipy import stats

y_true = df['Class']
X = df.drop(columns='Class')

X_train, X_test, y_train, y_test = train_test_split(X,y_true,test_size = 0.33, random_state=0)

v11_mean = X_train['V11'].mean()
v11_std = X_train['V11'].std()
norm = stats.norm(

    loc = v11_mean, 
    scale = v11_std 

)

print(v11_mean, v11_std)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.gridspec as gridspec

var_name = 'V11'

fig, ax = plt.subplots(1, 1)
x = np.linspace(norm.ppf(0.01),
                norm.ppf(0.99), 100)
ax.plot(x, norm.pdf(x),
       'r-', lw=5, alpha=0.6, label='norm pdf')
ax.hist(X_train[var_name].values, density=True, histtype='stepfilled', bins=100);
plt.show()



We can also look at the difference in distribution for some variables between fraudulent and non-fraudulent transactions:

In [ ]:
features=[f for f in df.columns if 'V' in f]
nplots=np.size(features)
plt.figure(figsize=(15,4*nplots))
gs = gridspec.GridSpec(nplots,1)
for i, feat in enumerate(features):
    ax = plt.subplot(gs[i])
    sns.distplot(X_train[feat][y_train==1], bins=30)
    sns.distplot(X_train[feat][y_train==0],bins=30)
    ax.legend(['fraudulent', 'non-fraudulent'],loc='best')
    ax.set_xlabel('')
    ax.set_title('Distribution of feature: ' + feat)


    

**Q.** Explain how these graphs could provide meaningful information about anomaly detection using a gaussian model. **[2 Marks]**



**All the features are modeled on a Gaussian Distribution and given a new data-point, the probability of the data-point is given by Gaussian/Normal Distribution Function. If the probability is below a particular threshold (which is set depending upon the performance of the model on the Validation Set), the new data-point is claimed to be an outlier or anomalous.**

## Part 2: Single Variable Gaussian Distribution: [7 Marks]
We'll start by making a prediction using **a single feature of our dataset at a time**. Let's explore the following scenarios:

1.   **One Gaussian:** First, we will fit a single gaussian distribution.

  *   Plot a ROC curve for each feature, fitting non-fraudulent transactions.
  *   Find the best feature to distinguish fraudulent transactions from non-fraudulent transactions based on AUC (Area under the ROC Curve)
  *  To make a prediction, we compute the probability that each transaction in the training set belongs to that distribution. If the probability is lower than a threshold, we consider that transaction a fraud. Find an optimal threshold that maximizes the F1 Score in the training set. Report peformance on the test set as well.

  


2.   **Two Gaussians:** In this case, we will fit a Gaussian to each class in the selected feature above — one gaussian to the feature of non-fraudulent transactions and one gaussian for fraudulent transactions. To make predictions, we use the probability of a transaction belonging to the fraudulent Gaussian distribution and non-fraudulent distribution. 

  * Build a classifier using this two-gaussian approach on the training set, and once complete, test performance on the test set.

Summarize your findings.
**upon the practices, non-fraudulent has better F1 score obviously. number of Components do affect performance of prediction like in single-gaussian case, it has a little higher f1 score than result in two gaussian distribution with nonfraudulence.**

In [ ]:
def estimateGaussian(X):
    """
     This function estimates the parameters of a Gaussian distribution using the data in X
    """
    
    m = X.shape[0]
    
    #compute mean of X
    sum_ = np.sum(X,axis=0)
    mu = 1/m *sum_
    
    # compute variance of X
    var = 1/m * np.sum((X - mu)**2,axis=0)
    
    return mu,var
mu, sigma2 = estimateGaussian(X)

def multivariateGaussian(X, mu, sigma2):
    """
    Computes the probability density function of the multivariate gaussian distribution.
    """
    k = len(mu)
    
    sigma2=np.diag(sigma2)
    X = X - mu.T
    p = 1/((2*np.pi)**(k/2)*(np.linalg.det(sigma2)**0.5))* np.exp(-0.5* np.sum(X @ np.linalg.pinv(sigma2) * X,axis=1))
    return p
p = multivariateGaussian(X, mu, sigma2)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve 

# model 

mu, sigma = estimateGaussian(X_train)
p = multivariateGaussian(X_test, mu, sigma)
gm = GaussianMixture(n_components=1, covariance_type='full', random_state=0,)

# One Gaussian 

a = 0

for i in list(features):

  #confirm the feature 
  print(i)
  #prepare dataset
  x_train = np.array(X_train[i])
  x_test = np.array(X_test[i])
  
  #evoluate model with picked feature 
  gm.fit(x_train.reshape(-1,1))
  p_gm = gm.score_samples(x_test.reshape(-1,1))
  fpr_sc, tpr_sc, _ = roc_curve(y_test, 1-p)
  fpr_gm, tpr_gm, _ = roc_curve(y_test, 1-p_gm)

  temp = auc(fpr_gm, tpr_gm)

  if (a<temp):
    a = temp
    featname = i

  plt.plot(fpr_sc, tpr_sc, linestyle = '--', label='Single Component')
  plt.plot(fpr_gm, tpr_gm, marker='.', label='Gaussian Mixture')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend()
  plt.show()


print('the best feature:',featname)
print('AUC is:', a)









In [ ]:
#One Gaussian continue 

def f1score(precision,recall):

  return(2*precision*recall/(precision+recall))


feature = 'V14'
X_train_v14 = np.array(X_train[feature])
X_test_v14 = np.array(X_test[feature])
gm.fit(X_train_v14.reshape(-1,1))


#p_gm by train set 
p_gm = gm.score_samples(X_train_v14.reshape(-1,1))

a = 0
for i in range(1,101):
  mn_gm = sorted(p_gm)[i] 
  precision = precision_score(y_train, p_gm < mn_gm) 
  recall = recall_score(y_train, p_gm < mn_gm) 
  #print('For a k of ',i,' the precision is ', '%.3f' % precision,' and the recall is ', '%.3f' % recall) #We print precision and recall three times
  temp = f1score(precision,recall)
  if(a<temp):
    a = temp 
    threshold = mn_gm

print('For training set')
print('the optimal f1 score:',a,'the threshold:',threshold)



#p_gm by test set 

p_gm = gm.score_samples(X_test_v14.reshape(-1,1))

a = 0
for i in range(1,101):
  mn_gm = sorted(p_gm)[i] 
  precision = precision_score(y_test, p_gm < mn_gm) 
  recall = recall_score(y_test, p_gm < mn_gm)
  #print('For a k of ',i,' the precision is ', '%.3f' % precision,' and the recall is ', '%.3f' % recall) #We print precision and recall three times
  temp = f1score(precision,recall)
  if(a<temp):
    a = temp 
    threshold = mn_gm

print('for testing set')
print('the optimal f1 score:',a,'the threshold:',threshold)



In [ ]:
# Two Gaussians
#nonfraudulent vs fraudulent 
var = 'V14'
mu, sigma = estimateGaussian(X_train)
p = multivariateGaussian(X_test, mu, sigma)
gm1 = GaussianMixture(n_components=2, covariance_type='full', random_state=0,)
#set the data
X_train_nonfrau = np.array(X_train[var][y_train==0])
X_train_frau = np.array(X_train[var][y_train==1])
#build p_gm by train set with nonfraudulent 
gm1.fit(X_train_nonfrau.reshape(-1,1))
#get test result with nonfraudulent
p_gm = gm1.score_samples(X_test_v14.reshape(-1,1))
a = 0
for i in range(1,101):
  mn_gm = sorted(p_gm)[i] #We sort the points by probability, as before
  precision = precision_score(y_test, p_gm < mn_gm) #Here, we compare y_test labels to our picks using precision
  recall = recall_score(y_test, p_gm < mn_gm) #Here, we compare y_test labels to our picks using recall
  #print('For a k of ',i,' the precision is ', '%.3f' % precision,' and the recall is ', '%.3f' % recall) #We print precision and recall three times
  temp = f1score(precision,recall)
  if(a<temp):
    a = temp 
    threshold = mn_gm

print('nonfraudulent transaction')
print('the optimal f1 score:',a,'the threshold:',threshold)
fpr_sc, tpr_sc, _ = roc_curve(y_test, 1-p)
fpr_gm, tpr_gm, _ = roc_curve(y_test, 1-p_gm)

plt.plot(fpr_sc, tpr_sc, linestyle = '--', label='Single Component')
plt.plot(fpr_gm, tpr_gm, marker='.', label='Gaussian Mixture')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()


#build p_gm by train set with fraudulent 
gm2 = GaussianMixture(n_components=2, covariance_type='full', random_state=0,)
gm2.fit(X_train_frau.reshape(-1,1))
#get test result with fraudulent
p_gm1 = gm2.score_samples(X_test_v14.reshape(-1,1))
a = 0 
for i in range(1,101):
  mn_gm = sorted(p_gm1)[i] #We sort the points by probability, as before
  precision = precision_score(y_test, p_gm1 < mn_gm) #Here, we compare y_test labels to our picks using precision
  recall = recall_score(y_test, p_gm1 < mn_gm) #Here, we compare y_test labels to our picks using recall
  #print('For a k of ',i,' the precision is ', '%.3f' % precision,' and the recall is ', '%.3f' % recall) #We print precision and recall three times
  temp = f1score(precision,recall)
  if(a<temp):
    a = temp 
    threshold = mn_gm
print('fraudulent transaction')
print('the optimal f1 score:',a,'the threshold:',threshold)

fpr_sc, tpr_sc, _ = roc_curve(y_test, 1-p)
fpr_gm, tpr_gm, _ = roc_curve(y_test, 1-p_gm1)

plt.plot(fpr_sc, tpr_sc, linestyle = '--', label='Single Component')
plt.plot(fpr_gm, tpr_gm, marker='.', label='Gaussian Mixture')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()



## Part 3: Multivariate and Mixture of Gaussian Distribution [7 Marks]
We want to build an outlier detection model that boosts our F1 score compared to the last part. To design your model, you can benefit from:

*   Any number of features you want (multivariate).
*   You can fit your Gaussians only on non-fraudulent transactions or to both classes.
*   To fit your model, you can take advantage of the Gaussian mixture model where you can set the number of components [help](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) (take a look at the hint below).

It is up to you how to design your model. Try at least 5 different models and report the F1 score for each one. If your model requires a threshold to classify transactions, draw a ROC curve for each model, and compare them with part 2.1. Try to maximize your F1 score. What kind of model works better? How many features? Which features? How many Gaussians? How many components? Summarize your findings with tables and plots. 


**HINT!**

Does it make sense to have more than two Gaussians (or a mixture with more than 2 components) in this kind of (binary) task?

 The answer is YES, and it depends on the distribution of data. Remember the tutorial, where we had 5 clusters and fit a mixture of 5 Gaussians to detect outliers. Consider the transaction is valid if it is close to each one of those five clusters, and it is not valid if it is not close to them! For example (figure below), assume we have a high volume of legitimate transactions in the morning until lunchtime (9 AM to 12 PM). We have fewer transactions during lunchtime. Then we have another pick between (1 PM to 4 PM). In this example, it is reasonable to have two valid clusters (compenents), one for the morning and another for the afternoon transactions.
 ![Example:](https://raw.githubusercontent.com/aps1070-2019/datasets/master/img/AMoW2.png)
 
 **As can be seen from the outcome of the program below, if we select those features whose distribution is basically the same between non-fraud and fraudulent, we can barely have a good result. However, if we choose the features in which the non-fraud and fraudulent are quite different from each other, we can get a good result.**
 **also because the results I got from the P3, the model (ie.X_train[['V12','V14','V21']]) with randomly picked n_components=4 has the maximum f1 score in that iteration.**


In [ ]:
for i in range(0,4):
  X_train_m1 = [X_train[['V10','V11','V3']],X_train[['V23','V24','V25']],X_train[['V15','V16','V17']],X_train[['V12','V14','V21']],X_train[['V2','V11','V14']]]
  X_test_m1 = [X_test[['V10','V11','V3']],X_test[['V23','V24','V25']],X_test[['V15','V16','V17']],X_test[['V12','V14','V21']],X_train[['V2','V11','V14']]]
  n_compo = np.random.randint(2,5)
  
  a = 0
  train_data_x = np.array(X_train_m1[i])
  test_data_x = np.array(X_test_m1[i])
  mu, sigma = estimateGaussian(train_data_x)
  p = multivariateGaussian(test_data_x, mu, sigma)
  gm = GaussianMixture(n_components = n_compo,
                      covariance_type = 'full', random_state=0, )
  gm.fit(train_data_x)
  p_gm = gm.score_samples(test_data_x) 

  print('the order of model is:', i)

  for i in range(1,101): 
    mn_gm = sorted(p_gm)[i]
    precision = precision_score(y_test, p_gm < mn_gm) 
    recall = recall_score(y_test, p_gm < mn_gm) 
     
    temp = f1score(precision,recall)
    if(a<temp):
     a = temp 
     threshold = mn_gm
    
    
  print('the f1 score is:',f1score(precision,recall), 'the threshold:', threshold)
  print('the picked component is:', n_compo)
  

    

  
  fpr_sc, tpr_sc, _ = roc_curve(y_test, 1-p)
  fpr_gm, tpr_gm, _ = roc_curve(y_test, 1-p_gm)
  plt.plot(fpr_sc, tpr_sc, linestyle = '--', label='Single Component')
  plt.plot(fpr_gm, tpr_gm, marker='.', label='Gaussian Mixture')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend()
  plt.show()

  print('--------------------') 
  print('--------------------') 